In [1]:
from logging import error
from web3.auto import w3
import json
import requests
import os
from dotenv import load_dotenv
from datetime import datetime as dt

load_dotenv()


Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


True

In [2]:
def init_contract(abi_path:str, contract_address:str):
    """ Initialize a contract given the abi and address """
    with open(abi_path) as json_file:
        abi = json.load(json_file)
    return w3.eth.contract(address=contract_address, abi=abi)



In [3]:
os.getenv("CHARITY_MAKER_ADDRESS")

'0xeaF2D0E9A31448cDC39ed43491651b90C4d80D38'

In [4]:
charity_contract = init_contract("CharityMakerABI.json", os.getenv("CHARITY_MAKER_ADDRESS"))


In [5]:
headers = {
    "Content-Type": "application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY"),
}

In [6]:
def convertDataToJSON(content:dict):
    data = {
        "pinataOptions": {"cidVersion": 1},
        "pinataContent": content,
    }
    return json.dumps(data)

def pinJSONtoIPFS(json):
    req = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS", data=json, headers=headers
    )
    ipfs_hash = req.json()["IpfsHash"]
    return ipfs_hash


In [7]:
def toDict(dictToParse):
    # convert any 'AttributeDict' type found to 'dict'
    parsedDict = dict(dictToParse)
    for key, val in parsedDict.items():
        # check for nested dict structures to iterate through
        if  'dict' in str(type(val)).lower():
            parsedDict[key] = toDict(val)
        # convert 'HexBytes' type to 'str'
        elif 'HexBytes' in str(type(val)):
            parsedDict[key] = val.hex()
    return parsedDict

def get_charityEventID_from_URI(event_URI: str):
    charity_event_reg_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"URI": event_URI})
    charity_event_registrations = charity_event_reg_filter.get_all_entries()
    charity_event_registrations_dict = toDict(charity_event_registrations[0])
    return charity_event_registrations_dict['args']['charityEventID']

def register_charity_event(event_name: str, event_recipient: str, funding_goal: int, start_date, end_date):

    # convert string start and end dates to datetime (if they aren't already datetime objects)
    if not isinstance(start_date, dt):
        start_date = dt.strptime(start_date, "%Y/%m/%d")  # 2020/01/26
    if not isinstance(end_date, dt):
        end_date = dt.strptime(end_date, "%Y/%m/%d")

    # check if start_date is in the past and if so make the start_date today instead
    # today = dt.strptime(dt.now(), "%Y/%m/%d")
    today = dt.now()
    if start_date < today:
        start_date = today
    
    # check if end date is after start_date and return error if not
    if start_date > end_date:
        return error(f"Error: End Date {end_date} is before Start Date {start_date}")

    # convert start and end datetimes to to unix timestamps
    start_date = int(start_date.timestamp())
    end_date = int(end_date.timestamp())

    # create charity_info dict
    charity_info = {
        "charityEventName" : event_name,
        # charityEventRecipient: event_recipient,
        "goalAmount" : funding_goal
    }

    # convert charity_info to json_charity_info (json format) and pin to IPFS via pinata api function
    json_charity_info = convertDataToJSON(charity_info)
    ipfs_link = pinJSONtoIPFS(json_charity_info)

    # create charity event in the block chain
    tx_hash = charity_contract.functions.registerCharityEvent(event_recipient, start_date, end_date, ipfs_link).transact({"from": w3.eth.accounts[0]})
    receipt  = w3.eth.waitForTransactionReceipt(tx_hash)
    
    # function registerCharityEvent(
    #     address payable _charityEventAddress,
    #     uint _startDate,
    #     uint _endDate,
    #     string memory _URI

    
    # lookup registerCharityEvent events, filtering by the newly created ipfs_link (URI) to cross reference and return the newly generated charity event id
    charity_event_id = get_charityEventID_from_URI(ipfs_link)

    return charity_event_id


In [8]:
# test register charity
event_id = register_charity_event("Dan's best charity#1", "0x6b36C3CBFcD41C747305e540266AEe5a867f948A", 200, "2020/11/15", "2021/11/15")


In [9]:
event_id

1

In [10]:
def update_charity_event_approval(charity_event_id: int, is_approved: bool):
    # call contract funtion to update approval
    tx_hash = charity_contract.functions.updateCharityEventApproval(charity_event_id, is_approved)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [11]:
# test update_charity_event_approval
update_charity_event_approval(1, True)

AttributeDict({'transactionHash': HexBytes('0xb3f6b822a946b36721dc40102ca87131489883601eb2fad0590139f444206312'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x5ea6c15fe0fd75bf9c0e441977b6b0e6138cc0f9c77f3e607376b5392541db6a'),
 'blockNumber': 126,
 'from': '0x13C0fAd72f393A9ea42AE90c7a868f509cF13CD6',
 'to': '0xeaF2D0E9A31448cDC39ed43491651b90C4d80D38',
 'gasUsed': 43522,
 'cumulativeGasUsed': 43522,
 'contractAddress': None,
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [13]:
charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": 1})
charity_events = charity_event_filter.get_all_entries()
charity_event_dict = toDict(charity_events[0])
charity_event_dict['args']['URI']

# sample http call
# https://gateway.pinata.cloud/ipfs/bafkreiew4vas3g3bi27bx4rblutplsholoaot24lsb3mzmhwwq3ibkunrm

'bafkreif3bgfrs63jtww4k5lcju3sr66owy7orywthqxdulh7gcbs3gyu7q'

In [14]:
# get JSON from IPFS
def getJSONfromPinata(ipfs_hash):
#     response = requests.get(f"https://gateway.pinata.cloud/ipfs/{charity_event_dict['args']['URI']}")
#     json_data = json.loads(response.text)
    response = requests.get(f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}")
    return json.loads(response.text)

In [16]:
json_data = getJSONfromPinata("bafkreiew4vas3g3bi27bx4rblutplsholoaot24lsb3mzmhwwq3ibkunrm")

In [17]:
type(json_data), json_data

(dict, {'charityEventName': "Dan's best charity#3", 'goalAmount': 200})

In [23]:
### testing solidity getter function call
# charity_event_filter = charity_contract.events.charityEventRegistration.createFilter(fromBlock="0x0", argument_filters={"charityEventID": charity_event_id})
# charity_events = charity_event_filter.get_all_entries()
# charity_event_dict = toDict(charity_events[0])
charity_event_filter = charity_contract.functions.getCharityEventInfo(1).call()
# charity_events = charity_event_filter.get_all_entries()
#charity_event_dict = toDict(charity_events[0])

#charity_event_dict['args']['charityEventAddress']

# charityBook

In [33]:
# test output
print(f"{charity_event_filter[5]} {type(charity_event_filter[5])}")

bafkreif3bgfrs63jtww4k5lcju3sr66owy7orywthqxdulh7gcbs3gyu7q <class 'str'>


In [55]:
### testing solidty donations event call
donations = charity_contract.events.Donate.createFilter(fromBlock="0x0", argument_filters={"charityEventID": 1})
donations_filter = donations.get_all_entries()
# donations_list = toDict(donations_filter[0])
# donations_filter
# donations_dict

donations_list = []
for donation in donations_filter:
    donations_list.append(toDict(donation))
    
donations_list[-1]


{'args': {'donorName': 'Cheapskate',
  'donorAmount': 250000000000000000,
  'charityEventID': 1},
 'event': 'Donate',
 'logIndex': 0,
 'transactionIndex': 0,
 'transactionHash': '0x8b64d3fc02d5e7cf7a98cb3b9189118535d0bee40237889bfda98df5905d7ff0',
 'address': '0xeaF2D0E9A31448cDC39ed43491651b90C4d80D38',
 'blockHash': '0x6170bea7c16033ebc8dd3c0a5725c24143f2515a3e93b027fa9365732d08531c',
 'blockNumber': 129}

In [57]:
### test summing donations
total_donations = 0
for donation in donations_list:
    total_donations += donation['args']['donorAmount']

total_donations

1750000000000000000

In [58]:
def get_total_donations(charity_event_id):
    donations = charity_contract.events.Donate.createFilter(fromBlock="0x0", argument_filters={"charityEventID": charity_event_id})
    donations_filter = donations.get_all_entries()

    # loop through solidity returned list of objects and convert to list of dicts
    donations_list = []
    for donation in donations_filter:
        donations_list.append(toDict(donation))

    # loop through donations_list and add up donorAmount from each donation
    total_donations = 0
    for donation in donations_list:
        total_donations += donation['args']['donorAmount']
        
    return total_donations

In [59]:
def get_charity_event(charity_event_id):
#     donations = charity_contract.events.Donate.createFilter(fromBlock="0x0", argument_filters={"charityEventID": charity_event_id})
#     donations_filter = charity_event_filter.get_all_entries()
#     donations_dict = toDict(donations_filter[0])

    solidity_info = charity_contract.functions.getCharityEventInfo(charity_event_id).call()

    ipfs_hash = solidity_info[5]
    charity_ipfs_data = getJSONfromPinata(ipfs_hash)

    # [1,
    #  '0x6b36C3CBFcD41C747305e540266AEe5a867f948A',
    #  1605475468,
    #  1636956000,
    #  True,
    #  'bafkreif3bgfrs63jtww4k5lcju3sr66owy7orywthqxdulh7gcbs3gyu7q']
    
    charity_event_info = {
        "charityEventName": charity_ipfs_data['charityEventName'],
        "charityEventID": charity_event_id,
        "charityEventAddress": solidity_info[1],
        "startDate": dt.utcfromtimestamp(solidity_info[2]).strftime('%Y/%m/%d'),
        "endDate": dt.utcfromtimestamp(solidity_info[3]).strftime('%Y/%m/%d'),
        "goalAmount": charity_ipfs_data['goalAmount'],
        "totalDonations": get_total_donations(charity_event_id),
        "isApproved": solidity_info[4],
        "ipfsHash": solidity_info[5],
        "ipfsLink": f"https://gateway.pinata.cloud/ipfs/{ipfs_hash}"
    }

    return charity_event_info

In [60]:
event3_info = get_charity_event(1)

In [61]:
event3_info

{'charityEventName': "Dan's best charity#1",
 'charityEventID': 1,
 'charityEventAddress': '0x6b36C3CBFcD41C747305e540266AEe5a867f948A',
 'startDate': '2020/11/15',
 'endDate': '2021/11/15',
 'goalAmount': 200,
 'totalDonations': 1750000000000000000,
 'isApproved': True,
 'ipfsHash': 'bafkreif3bgfrs63jtww4k5lcju3sr66owy7orywthqxdulh7gcbs3gyu7q',
 'ipfsLink': 'https://gateway.pinata.cloud/ipfs/bafkreif3bgfrs63jtww4k5lcju3sr66owy7orywthqxdulh7gcbs3gyu7q'}